In [1]:
import pandas as pd

In [2]:
data_dir = "./data/clusterdata-2011-2/"
task_events_header = ['timestamp','missing_info', 'job_id','task_index','machine_id','event_type',
                      'user_name','sched_class','priority','cpu_req','ram_req','space_req','diff_machine']

In [4]:
df = pd.read_csv(data_dir+"task_events/part-00000-of-00500.csv.gz",header=None)
# df.columns = task_events_header

In [5]:
# df[df['job_id'] == 3418309]
df['event_type'].value_counts()


0    201491
1    189994
4     43237
5     10505
2      4338
3       581
Name: event_type, dtype: int64

In [16]:
581+43237+10505+4338

58661